In [1]:
#!/usr/bin/env python3
'''
Created on Thu June 5 2018

__author__ = "Zhejian Peng"
__license__ = MIT
__credits__ = Lehman Brothers, John Hull's 'Option'
__version__ = "1.0.1"
__maintainer__ = "Zhejian Peng"
__email__ = 'zhejianpeng@yahoo.com'
__status__ = "Developing"

__update__ =
'''
# %%
# Import library
import numpy as np
import pandas as pd
from scipy.optimize import newton

# %%
# Define inputs of a cds contract
yieldcurveTenor = [0.5, 1, 2, 3, 4, 5]
yieldcurveRate = [0.01350, 0.01430, 0.0190, 0.02470, 0.02936, 0.03311]
creditcurveTenor = [1, 3, 5, 7]
creditcurveSP = [0.99, 0.98, 0.95, 0.92]
cdsTenors = [1, 2, 3, 4, 5]
cdsSpreads = [0.0110, 0.0120, 0.0130, 0.0140, 0.0150]
premiumFrequency = 4
defaultFrequency = 12
accruedPremium = True
recoveryRate = 0.40

In [112]:
class cds:
    def __init__(self, creditcurve, creditcurveLength, yieldcurve, yieldcurveLength,
                 cdsTenor, premiumFrequency, defaultFrequency, accruedPremium, recoveryRate, spread):
        '''
        double *creditcurve;
        int creditcurveLength;
        double *yieldcurve;
        int yieldcurveLength;
        double cdsTenor;
        double premiumFrequency;
        double defaultFrequency;
        int accruedPremium;
        double recoveryRate;
        double spread;
        '''
        self.creditcurve = creditcurve
        self.creditcurveLength = creditcurveLength
        self.yieldcurve = yieldcurve
        self.yieldcurveLength = yieldcurveLength
        self.cdsTenor = cdsTenor
        self.premiumFrequency = premiumFrequency
        self.defaultFrequency = defaultFrequency
        self.accruedPremium = accruedPremium
        self.recoveryRate = recoveryRate
        self.spread = spread
        
    def getDiscountFactor(self, yieldcurveTenor, yieldcurveRate, t):
        '''
        yieldcurveRate: []
        yieldcurveTenor: []
        '''
        result = -1
        min_time_index = 0
        max_time_index = len(yieldcurveTenor) - 1

        if t < 0:
            result = - 1
        elif t == 0:
            result = 1.0
        elif t > 0 and t <= yieldcurveTenor[min_time_index]:
            result = np.exp(-t*yieldcurveRate[0])
        elif t >= yieldcurveTenor[max_time_index]:
            result = np.exp(-t*yieldcurveRate[-1])
        else:
            for i in range(max_time_index):
                # print(t, t >= yieldcurveTenor[i+1] and t < yieldcurveTenor[i+2])
                if t >= yieldcurveTenor[i] and t < yieldcurveTenor[i+1]:
                   # print(yieldcurveTenor[i], yieldcurveTenor[i+1])
                    yield_interpolated = yieldcurveRate[i] + (yieldcurveRate[i+1] - yieldcurveRate[i]) / \
                    (yieldcurveTenor[i+1]-yieldcurveTenor[i]) * (t-yieldcurveTenor[i])
                    result = np.exp(-t*yield_interpolated)
        return result
    
    def getSurvivalProbability(self, creditcurveTenor, creditcurveSP, t):
        result = -1
        min_time_index = 0
        max_time_index = len(creditcurveTenor) - 1
        if t < 0:
            result = -1
        elif t == 0:
            result = 1
        elif t > 0 and t <= creditcurveTenor[min_time_index]:
            h = -np.log(creditcurveSP[0] / creditcurveTenor[min_time_index])
            result = np.exp(-h*t)
        elif t == creditcurveTenor[max_time_index]:
            result = creditcurveSP[-1]
        elif t > creditcurveTenor[max_time_index]:
            h = 0
            if len(creditcurveTenor) == 1:
                h = h = - np.log(creditcurveSP[-1]) / creditcurveTenor[max_time_index]
            else: 
                h = - np.log(creditcurveSP[-1]/creditcurveSP[-2]) / \
                        (creditcurveTenor[-1]-creditcurveTenor[-2])
                result = creditcurveSP[-1] * np.exp(-(t - creditcurveTenor[max_time_index])*h)
        else:  # where t is in between min_time and max_time
            for i in range(max_time_index):
                if t >= creditcurveTenor[i] and t < creditcurveTenor[i+1]:
                    h = -np.log(creditcurveSP[i+1]/creditcurveSP[i]) / \
                        (creditcurveTenor[i+1]-creditcurveTenor[i])
                    result = creditcurveSP[i] * \
                        np.exp(-(t-creditcurveTenor[i])*h)
                        # print('HERE,', creditcurveSP[i])
        return result


In [101]:
def getDiscountFactor(yieldcurveTenor, yieldcurveRate, t):
    '''
    yieldcurveRate: []
    yieldcurveTenor: []
    '''
    result = -1
    min_time_index = 0
    max_time_index = len(yieldcurveTenor) - 1

    if t < 0:
        result = - 1
    elif t == 0:
        result = 1.0
    elif t > 0 and t <= yieldcurveTenor[min_time_index]:
        result = np.exp(-t*yieldcurveRate[0])
    elif t >= yieldcurveTenor[max_time_index]:
        result = np.exp(-t*yieldcurveRate[-1])
    else:
        for i in range(max_time_index):
            # print(t, t >= yieldcurveTenor[i+1] and t < yieldcurveTenor[i+2])
            if t >= yieldcurveTenor[i] and t < yieldcurveTenor[i+1]:
               # print(yieldcurveTenor[i], yieldcurveTenor[i+1])
                yield_interpolated = yieldcurveRate[i] + (yieldcurveRate[i+1] - yieldcurveRate[i]) / \
                (yieldcurveTenor[i+1]-yieldcurveTenor[i]) * (t-yieldcurveTenor[i])
                result = np.exp(-t*yield_interpolated)
    return result
        

In [100]:
# Unit Test getDiscountFactor
def unit_test_getDiscountFactor(t, yield_true):
    epsilon = 10**-6
    rate = getDiscountFactor(yieldcurveTenor, yieldcurveRate, t)
    print('test t =', t, ', yieldRate = ', rate)
    error = rate - yield_true
    # print(error)
    assert((abs(error) < epsilon))


test_t = [-1, 0, 1, 2, 3, 4, 5, 6, 1.5]
yield_ans = [-1, 1.000000, 0.985802, 0.962713,
             0.928579, 0.889194, 0.847427, 0.819829, 0.975334]
print('TEST: cds.getDiscountFactor()')
for i, j in zip(test_t, yield_ans):
    # print(i, j)
    unit_test_getDiscountFactor(i, j)

TEST: cds.getDiscountFactor()
test t = -1 , yieldRate =  -1
test t = 0 , yieldRate =  1.0
test t = 1 , yieldRate =  0.98580175937
test t = 2 , yieldRate =  0.962712940891
test t = 3 , yieldRate =  0.928578831315
test t = 4 , yieldRate =  0.889193861778
test t = 5 , yieldRate =  0.847427490771
test t = 6 , yieldRate =  0.819828587672
test t = 1.5 , yieldRate =  0.975334295081


In [113]:
def getSurvivalProbability(creditcurveTenor, creditcurveSP, t):
    result = -1
    min_time_index = 0
    max_time_index = len(creditcurveTenor) - 1
    if t < 0:
        result = -1
    elif t == 0:
        result = 1
    elif t > 0 and t <= creditcurveTenor[min_time_index]:
        h = -np.log(creditcurveSP[0] / creditcurveTenor[min_time_index])
        result = np.exp(-h*t)
    elif t == creditcurveTenor[max_time_index]:
        result = creditcurveSP[-1]
    elif t > creditcurveTenor[max_time_index]:
        h = 0
        if len(creditcurveTenor) == 1:
            h = h = - np.log(creditcurveSP[-1]) / creditcurveTenor[max_time_index]
        else: 
            h = - np.log(creditcurveSP[-1]/creditcurveSP[-2]) / \
                    (creditcurveTenor[-1]-creditcurveTenor[-2])
            result = creditcurveSP[-1] * np.exp(-(t - creditcurveTenor[max_time_index])*h)
    else:  # where t is in between min_time and max_time
        for i in range(max_time_index):
            if t >= creditcurveTenor[i] and t < creditcurveTenor[i+1]:
                h = -np.log(creditcurveSP[i+1]/creditcurveSP[i]) / \
                    (creditcurveTenor[i+1]-creditcurveTenor[i])
                result = creditcurveSP[i] * \
                    np.exp(-(t-creditcurveTenor[i])*h)
                    # print('HERE,', creditcurveSP[i])
    return result
        

In [111]:
def unit_test_getSurvivalProbability(t, sp_true):
    epsilon = 10**-6
    sp = getSurvivalProbability(creditcurveTenor, creditcurveSP, t)
    print('test t =', t, ', survival probability = ', sp)
    error = sp - sp_true
    # print(error)
    assert((abs(error) < epsilon))


print('\nTEST: cds.getSurvivalProbability()')

test_t = [-1, 0, 1, 3, 5, 7, 10, 5.5]
sp_ans = [-1, 1, 0.990000, 0.980000, 0.950000, 0.920000, 0.876767, 0.942410]
for i, j in zip(test_t, sp_ans):
    unit_test_getSurvivalProbability(i, j)


TEST: cds.getSurvivalProbability()
test t = -1 , survival probability =  -1
test t = 0 , survival probability =  1
test t = 1 , survival probability =  0.99
test t = 3 , survival probability =  0.98
test t = 5 , survival probability =  0.95
test t = 7 , survival probability =  0.92
test t = 10 , survival probability =  0.876766929501
test t = 5.5 , survival probability =  0.942409511747


In [133]:
# double calculatePremiumLeg(double *creditcurve, int ncreditcurve, 
# double *yieldcurve, int nyieldcurve, double cdsMaturity, int numberPremiumPerYear,int accruedPremiumFlag, 
# double spread, double h) {}
def calculatePremiumLeg(creditcurveTenor, creditcurveSP, yieldcurveTenor, yieldcurveRate, cdsMaturity, 
                        num_premium_year, accruedPremiumFlag, spread, h):
    max_time_index = len(creditcurveTenor) - 1
    if max_time_index > 0 and cdsMaturity <= creditcurveTenor[max_time_index]:
            annuity = 0
            accruedPremium = 0
            N = int(cdsMaturity*num_premium_year)
            for n in range(1, N+1):
                tn = n / num_premium_year
                tnm1 = (n-1) / num_premium_year
                dt = 1.0 / num_premium_year
                annuity += dt * \
                    getDiscountFactor(yieldcurveTenor, yieldcurveRate, tn)*getSurvivalProbability(creditcurveTenor, creditcurveSP, tn)
                if accruedPremiumFlag:
                    accruedPremium += 0.5*dt*getDiscountFactor(yieldcurveTenor, yieldcurveRate, tn)*(
                        getSurvivalProbability(creditcurveTenor, creditcurveSP, tnm1) - getSurvivalProbability(creditcurveTenor, creditcurveSP, tn))
            # print('HERE', spread)
            return spread*(annuity+accruedPremium)
    else:  # When the cds maturity is beyond our current credit curve, we need to estimate the survival probability for payment beyond credit curve
        annuity = 0
        accruedPremium = 0
        N = int(cdsMaturity*num_premium_year)
        M = creditcurveTenor[max_time_index] * num_premium_year if max_time_index > 0 else 0

        for n in range(1, N+1):
            if n <= M:
                tn = n/num_premium_year
                tnm1 = (n-1)/num_premium_year
                dt = 1.0 / num_premium_year

                annuity += dt * getDiscountFactor(yieldcurveTenor, yieldcurveRate, tn) * \
                getSurvivalProbability(creditcurveTenor, creditcurveSP, tn)
                if(accruedPremiumFlag):
                    accruedPremium += 0.5*dt*getDiscountFactor(yieldcurveTenor, yieldcurveRate,tn)*(
                        getSurvivalProbability(creditcurveTenor, creditcurveSP,tnm1) - \
                        getSurvivalProbability(creditcurveTenor, creditcurveSP,tn))
            else:
                tn = n/num_premium_year
                tnm1 = (n-1)/num_premium_year
                tM = M / num_premium_year
                dt = 1.0 / num_premium_year

                survivalProbability_n = getSurvivalProbability(creditcurveTenor, creditcurveSP, tM) * \
                np.exp(-h*(tn - tM))
                survivalProbability_nm1 = 0
                if tnm1 <= tM:
                    survivalProbability_nm1 = getSurvivalProbability(creditcurveTenor, creditcurveSP, tnm1)
                else:
                    survivalProbability_nm1 = getSurvivalProbability(creditcurveTenor, creditcurveSP, tM)*\
                    np.exp(-h*(tnm1 - tM))

                annuity += dt * getDiscountFactor(yieldcurveTenor, yieldcurveRate, tn)*survivalProbability_n
                if accruedPremiumFlag:
                    accruedPremium += 0.5*dt * getDiscountFactor(yieldcurveTenor, yieldcurveRate , tn)*\
                    (survivalProbability_nm1-survivalProbability_n)

        return spread*(annuity+accruedPremium)

In [134]:
def unit_test_calculatePremiumLeg(param, pl_ture):
    epsilon = 10**-6
    maturity, spread, accruedPremium, h = param
    pl = calculatePremiumLeg(creditcurveTenor,creditcurveSP, yieldcurveTenor, yieldcurveRate, 
                             maturity, 4, accruedPremium, spread,h)
    print('test param =', param, ', premium leg = ', pl)
    error = pl - pl_ture
    assert(abs(error) < epsilon)


param = [[4, 0.02, True, 0.01], [-1, 0.02,
                                 True, 0.01], [4, 0.02, False, 0.01], [4, 0.1, True, 0.1], [8, 0.1, True, 0.1]]
pl_ans = [0.075030, 0, 0.074947, 0.375151623674, 0.678580]

print('\nTEST: cds.calculatePremiumLeg()')
for i, j in zip(param, pl_ans):
    unit_test_calculatePremiumLeg(i, j)


TEST: cds.calculatePremiumLeg()
test param = [4, 0.02, True, 0.01] , premium leg =  0.0750303247347
test param = [-1, 0.02, True, 0.01] , premium leg =  0.0
test param = [4, 0.02, False, 0.01] , premium leg =  0.0749474288233
test param = [4, 0.1, True, 0.1] , premium leg =  0.375151623674
test param = [8, 0.1, True, 0.1] , premium leg =  0.678580377742


In [259]:
# double calculateDefaultLeg(double *creditcurve, int ncreditcurve, double *yieldcurve, 
# int nyieldcurve, double cdsMaturity, int numberDefaultIntervalPerYear, double recoveryRate, double h) {

def calculateDefaultLeg(creditcurveTenor, creditcurveSP, yieldcurveTenor, yieldcurveRate, 
                        cdsMaturity, num_default_year, recoveryRate, h):
    # spread = self.spread
    max_time_index = len(creditcurveTenor) - 1
    # accruedPremiumFlag = self.accruedPremium  # True or False
    # print(cdsMaturity, recoveryRate, h, num_default_year)
    if max_time_index > 0 and cdsMaturity <= creditcurveTenor[max_time_index]:
        annuity = 0
        N = int(cdsMaturity * num_default_year)
        for n in range(1, N+1):
            tn = n / num_default_year
            tnm1 = (n-1) / num_default_year
            annuity += getDiscountFactor(yieldcurveTenor, yieldcurveRate, tn)*(
                getSurvivalProbability(creditcurveTenor, creditcurveSP, tnm1) - \
                getSurvivalProbability(creditcurveTenor, creditcurveSP, tn))
        return (1-recoveryRate)*annuity
    else:  # cdsMaturity > creditcurveTenor[max_time_index]
        annuity = 0
        N = int(cdsMaturity*num_default_year)
        M = creditcurveTenor[max_time_index] * num_default_year if max_time_index > 0 else 0

        for n in range(1, N+1):
            if n <= M:
                tn = n / num_default_year
                tnm1 = (n-1) / num_default_year
                annuity += getDiscountFactor(yieldcurveTenor, yieldcurveRate, tn)*(
                    getSurvivalProbability(creditcurveTenor, creditcurveSP, tnm1) - \
                    getSurvivalProbability(creditcurveTenor, creditcurveSP,tn))
            else:  # n > m
                tM = M / num_default_year
                tn = n / num_default_year
                tnm1 = (n-1) / num_default_year

                survivalProbability_n = getSurvivalProbability(creditcurveTenor, creditcurveSP, tM)*\
                                        np.exp(-h*(tn-tM))
                if tnm1 <= tM:
                    survivalProbability_nm1 = getSurvivalProbability(creditcurveTenor, creditcurveSP, tnm1)
                else:
                    survivalProbability_nm1 = getSurvivalProbability(creditcurveTenor, creditcurveSP,  tM) * np.exp(-h*(tnm1 - tM))
                annuity += getDiscountFactor(yieldcurveTenor, yieldcurveRate, tn) * (survivalProbability_nm1 - survivalProbability_n)
                # print('HERE', annuity)

        return (1-recoveryRate)*annuity


In [260]:
# %% Unit Test cds.calculateDefaultLeg()
def unit_test_calculateDefaultLeg(param, dl_true):
    epsilon = 10**-6
    maturity, defaultFrequency, recoveryRate, h = param
    dl = calculateDefaultLeg(creditcurveTenor, creditcurveSP, yieldcurveTenor, yieldcurveRate, 
                        maturity,defaultFrequency, recoveryRate, h)
    print('test param = ', param, ', default leg =', dl)
    error = dl - dl_true
    assert(abs(error) < epsilon)

    
param = [[4, 12, 0.4, 0.01], [0, 0, 0, 0],
         [-1, 12, 0.4, 0.01], [8, 12, 0.4, 0.01], [10, 12, 0.5, 0.01], [3.4, 100, 0.6, 0.1]]
pl_ans = [0.019947, 0, 0, 0.046710, 0.045612, 0.010053]

print('\nTEST: cds.calculateDefaultLeg()')
for i, j in zip(param, pl_ans):
    unit_test_calculateDefaultLeg(i,j)


TEST: cds.calculateDefaultLeg()
test param =  [4, 12, 0.4, 0.01] , default leg = 0.019946784292
test param =  [0, 0, 0, 0] , default leg = 0
test param =  [-1, 12, 0.4, 0.01] , default leg = 0.0
test param =  [8, 12, 0.4, 0.01] , default leg = 0.0467101054391
test param =  [10, 12, 0.5, 0.01] , default leg = 0.045611939249
test param =  [3.4, 100, 0.6, 0.1] , default leg = 0.0100527161735


In [261]:
yieldcurveTenor = [0.5, 1, 2, 3, 4, 5]
yieldcurveRate = [0.01350, 0.01430, 0.0190, 0.02470, 0.02936, 0.03311]
creditcurveTenor = [1, 3, 5, 7]
creditcurveSP = [0.99, 0.98, 0.95, 0.92]
cdsTenors = [1, 2, 3, 4, 5]
cdsSpreads = [0.0110, 0.0120, 0.0130, 0.0140, 0.0150]
premiumFrequency = 4
defaultFrequency = 12
accruedPremium = True
recoveryRate = 0.40
global cdsMaturity 
global spread
def objfunFindHazardRate(h):
    # print(cdsMaturity)
    premLeg = calculatePremiumLeg(creditcurveTenor, creditcurveSP, yieldcurveTenor, yieldcurveRate, cdsMaturity, premiumFrequency, 
                                 accruedPremium, spread,h)
    defaultLeg = calculateDefaultLeg(creditcurveTenor, creditcurveSP, yieldcurveTenor, yieldcurveRate, cdsMaturity, defaultFrequency, 
                                 recoveryRate, h)
    return premLeg - defaultLeg
    

In [262]:
# void bootstrapCreditDefaultSwapSpreads
# (double *yieldcurve, int *nyieldcurve, double *cdsTenors, int *ncdsTenors, double *spreads, 
# int *numberPremiumPerYear, int *numberDefaultIntervalPerYear, int *accruedPremiumFlag, 
# double *recoveryRate, double *output, int * warningFlag) {

def bootstrapCDSspread(yieldcurveTenor, yieldcurveRate, cdsTenors, cdsSpreads, premiumFrequency, defaultFrequency, 
                       accruedPremium, recoveryRate):
    yieldcurveLength = len(yieldcurveTenor)
    cdsTenorsLength = len(cdsTenors)
    
    newcreditcurveLength = 0
    newcreditcurve = []
    survprob = [None]*cdsTenorsLength
    hazardRate = [None]*cdsTenorsLength
    global creditcurveSP 
    creditcurveSP = []
    global creditcurveTenor 
    creditcurveTenor = []
    for i in range(cdsTenorsLength):
        global cdsMaturity 
        cdsMaturity = cdsTenors[i]
        global spread
        spread = cdsSpreads[i]
        # print(cdsMaturity, spread)
        h = newton(objfunFindHazardRate, 0.01)
        hazardRate[i] = h
        if i==0:
            survprob[i] = np.exp(-hazardRate[i]*cdsTenors[i])
        else:
            survprob[i] = survprob[i-1]*np.exp(-hazardRate[i]*(cdsTenors[i]-cdsTenors[i-1]))
        creditcurveTenor.append(cdsTenors[i])
        creditcurveSP.append(survprob[i])
    return hazardRate, survprob
        

In [263]:
result = bootstrapCDSspread(yieldcurveTenor, yieldcurveRate, cdsTenors, cdsSpreads, premiumFrequency, defaultFrequency, 
                       accruedPremium, recoveryRate)

In [264]:
result

([0.018311376780972728,
  0.019968032128537042,
  0.026995336307376588,
  0.028858240300829317,
  0.032750251934410245],
 [0.98185525782566352,
  0.96244398789389574,
  0.93681004393793976,
  0.91016171586524852,
  0.88083651530873952])

In [265]:
# Test on Lehman's result:
day_count = [0.261111, 0.252778, 0.252778, 0.252778, 0.252778, 0.252778, 0.252778, 0.252778, 0.255556, 0.252778,
             0.250000, 0.255556, 0.255556, 0.252778, 0.250000, 0.255556, 0.255556]
days = np.cumsum(day_count)
days

array([ 0.261111,  0.513889,  0.766667,  1.019445,  1.272223,  1.525001,
        1.777779,  2.030557,  2.286113,  2.538891,  2.788891,  3.044447,
        3.300003,  3.552781,  3.802781,  4.058337,  4.313893])

In [266]:
cdsTenors

[1, 2, 3, 4, 5]

In [267]:
cdsSpreads

[0.011, 0.012, 0.013, 0.014, 0.015]

In [268]:
# Interpolate CDS Spread
def interpolatedSpread(cdsTenors, cdsSpreads, t):
    result = -1
    min_time_index = 0
    max_time_index = len(cdsTenors) - 1
    if t < 0:
        result = -1
    elif t == 0:
        result = 0.01
    elif t < cdsTenors[min_time_index]:
        result = 0.01 + (cdsSpreads[1] - cdsSpreads[0]) / (cdsTenors[1] - cdsTenors[0]) * (t)
    elif t >= cdsTenors[max_time_index]:
        result = cdsSpreads[max_time_index]
    else: # in between
        for i in range(max_time_index):
            if t>= cdsTenors[i] and t<cdsTenors[i+1]:
                result = cdsSpreads[i] + (cdsSpreads[i+1] - cdsSpreads[i]) / (cdsTenors[i+1] - cdsTenors[i]) * (t - cdsTenors[i])
                
    return result
        
inter_spread=[]            
for i in days:
    inter_spread.append(interpolatedSpread(cdsTenors, cdsSpreads, i))

In [274]:
inter_spread

[0.010261111,
 0.010513889,
 0.010766667000000001,
 0.011019444999999999,
 0.011272223,
 0.011525001,
 0.011777779,
 0.012030557000000001,
 0.012286113,
 0.012538891,
 0.012788891,
 0.013044446999999999,
 0.013300002999999999,
 0.013552781,
 0.013802781,
 0.014058337000000001,
 0.014313892999999999]

In [272]:
result = bootstrapCDSspread(yieldcurveTenor, yieldcurveRate, days, inter_spread, premiumFrequency, 200, 
                       accruedPremium, recoveryRate)

In [273]:
result

([0.017082623419957175,
  0.017503447273633604,
  0.030837442075050743,
  0.019012775939841364,
  0.020792741462201402,
  0.021587162034757041,
  0.022474476856375089,
  0.023361053793171202,
  0.024306326245793472,
  0.025171082849043479,
  0.02603114753358492,
  0.027096462133392278,
  0.028046540545934164,
  0.028936080974079747,
  0.029824340362983939,
  0.031024156095347682,
  0.032004059557927872],
 [0.99554947219764445,
  0.99115440722485315,
  0.9834583663181462,
  0.97874319378541152,
  0.97361246607307206,
  0.96831416578106122,
  0.96282871769842338,
  0.95715981399533101,
  0.95123272126702996,
  0.94519953068761886,
  0.9390683454416977,
  0.93258807590511472,
  0.92592768413938431,
  0.91917978288136126,
  0.91235178689851382,
  0.90514688800051191,
  0.89777403772335496])

In [ ]:
newton()